In [1]:
import numpy as np
from dipy.io.streamline import load_trk
from dipy.viz import window, actor
import math
from nibabel import trackvis as tv
from dipy.tracking.streamline import Streamlines
from dipy.segment.clustering import QuickBundles
from dipy.io.pickles import save_pickle
from dipy.data import get_data
from dipy.segment.metric import mdf
from dipy.segment.metric import AveragePointwiseEuclideanMetric
import matplotlib.pyplot as plt
import pickle
from dipy.io.streamline import load_trk, save_trk
import nibabel

In [2]:
def show_streamlines(streamlines): #function to visualize streamlines
    
    ren = window.Renderer()
    ren.add(actor.line(streamlines))
    window.show(ren)

In [3]:
def show_multiple_streamlines(streamline1,streamline2): #function to visualize streamlines
    
    ren = window.Renderer()
    ren.add(actor.line(streamline1))
    ren.add(actor.line(streamline2,colors=(0,1,1)))
    window.show(ren)

 load brain (tractogram)

In [4]:
# brain, b_hr = load_trk("brain.trk")
brain=[]

load Arcuate Fasciculus left fiber bundle (AF_L)

In [5]:
# bundle, bn_hr = load_trk("AF_L.trk")
# brain.append(bundle)

In [6]:
file = nibabel.streamlines.load("SLF_I_right.tck")
save_trk("SLF_I_right.trk",file.streamlines, np.eye(4))
bundle, bn_hr = load_trk("SLF_I_right.trk")
brain.append(bundle[:50])

In [7]:
file = nibabel.streamlines.load("OR_left.tck")
save_trk("OR_left.trk",file.streamlines, np.eye(4))
bundle, bn_hr = load_trk("OR_left.trk")
brain.append(bundle[:50])

In [8]:
file = nibabel.streamlines.load("ICP_left.tck")
save_trk("ICP_left.trk",file.streamlines, np.eye(4))
bundle, bn_hr = load_trk("ICP_left.trk")
brain.append(bundle[:50])

In [9]:
file = nibabel.streamlines.load("FX_right.tck")
save_trk("FX_right.trk",file.streamlines, np.eye(4))
bundle, bn_hr = load_trk("FX_right.trk")
brain.append(bundle[:50])

In [10]:
# cst, bn_hr = load_trk("CST_recognized.trk")

let's visualize them

In [11]:
# first brain

#show_streamlines(brain)
show_multiple_streamlines(brain[1953:1954], brain[1950+81:1950+82])

ValueError: need at least one array to concatenate

In [11]:
# now bundle
brain = np.array(brain).flatten()


/Users/rp7/anaconda3/lib/python3.6/site-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \


In [12]:
show_streamlines(brain)

In [93]:
(bundle[0].shape)
len(bundle[0])

331

In [94]:
# get labels

def getLabels(brain, bundle):
    labels = np.zeros(len(brain))
    npbrain = np.array(brain)
    for i in range(len(bundle)):
        for j in range(len(brain)):
            if((npbrain[j].shape[0]==bundle[i].shape[0])):
    #         print(i,j)
                if np.allclose(npbrain[j], bundle[i]):
                    labels[j] = 1
                    break
        if(i%100==0):
            print(i,end=" ")
    return labels

In [95]:
labels = np.zeros(len(brain))
l = getLabels(brain, brain[:50])
labels[l==l] = 0
l = getLabels(brain, brain[50:100])
labels[l==l] = 1
l = getLabels(brain, brain[100:150])
labels[l==l] = 2
l = getLabels(brain, brain[150:200])
labels[l==l] = 3

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 

In [53]:
labels = np.zeros(len(brain))
labels[:50] = 0
labels[50:100] = 1
labels[100:150] = 2
labels[150:] = 3

In [14]:
#Find Frechet distance
# From : https://gist.github.com/MaxBareiss/ba2f9441d9455b56fbc9

# Euclidean distance.
def euc_dist(pt1,pt2):
    return math.sqrt((pt2[0]-pt1[0])*(pt2[0]-pt1[0])+(pt2[1]-pt1[1])*(pt2[1]-pt1[1]))

def _c(ca,i,j,P,Q):
    if ca[i,j] > -1:
        return ca[i,j]
    elif i == 0 and j == 0:
        ca[i,j] = euc_dist(P[0],Q[0])
    elif i > 0 and j == 0:
        ca[i,j] = max(_c(ca,i-1,0,P,Q),euc_dist(P[i],Q[0]))
    elif i == 0 and j > 0:
        ca[i,j] = max(_c(ca,0,j-1,P,Q),euc_dist(P[0],Q[j]))
    elif i > 0 and j > 0:
        ca[i,j] = max(min(_c(ca,i-1,j,P,Q),_c(ca,i-1,j-1,P,Q),_c(ca,i,j-1,P,Q)),euc_dist(P[i],Q[j]))
    else:
        ca[i,j] = float("inf")
    return ca[i,j]

""" Computes the discrete frechet distance between two polygonal lines
Algorithm: http://www.kr.tuwien.ac.at/staff/eiter/et-archive/cdtr9464.pdf
P and Q are arrays of 2-element arrays (points)
"""
def frechetDist(P,Q):
    ca = np.ones((len(P),len(Q)))
    ca = np.multiply(ca,-1)
    return _c(ca,len(P)-1,len(Q)-1,P,Q)

In [15]:
# Fast Dynamic time warping
# From: https://github.com/slaypni/fastdtw

import numpy as np
from fastdtw import fastdtw

In [16]:
from dipy.align.streamlinear import BundleMinDistanceMetric
from dipy.align.streamlinear import set_number_of_points
# BundleMinDistanceMetric().distance(cBrain3D[0],cBrain3D[1])

def bmd(a,b):
    static = set_number_of_points(a, len(a))
    moving = set_number_of_points(b, len(b))
    BMD = BundleMinDistanceMetric()
    BMD.setup(static, moving)
    x0 = np.array([0, 0, 0, 0, 0, 0, 1., 1., 1, 0, 0, 0])  # affine
    bmd_value = BMD.distance(x0.tolist())
    return bmd_value

In [17]:
# bmd(brain3D[0],brain3D[7])
# brain3D[1].shape

In [18]:
# Get distance between 2 curves
from scipy.spatial.distance import euclidean

def distanceBetweenCurves(c1, c2, algo='fastdtw'):
    if algo=='fastdtw':
        distance, path = fastdtw(c1,c2, radius = 1, dist = euclidean) # check out the 'radius' parameter to tune the distance
        return distance
    elif algo=='frechet':
        return frechetDist(c1,c2)
    elif algo=='bmd':
        return bmd(c1,c2)
    elif algo=='mdf':
        return mdf(c1,c2)

In [19]:
from sklearn.decomposition import PCA
def project2D(streamline):
    pca = PCA(n_components=2)
    newBundle = pca.fit_transform(streamline)
#     newStreamline = Streamlines([(np.hstack((newBundle,np.zeros((newBundle.shape[0])).reshape((-1,1)))))])
    newStreamline = Streamlines([newBundle])
    return newStreamline, pca.components_, pca.singular_values_

In [20]:
# print(pca.explained_variance_ratio_) 
# print(pca.singular_values_)

In [21]:
# st = Streamlines([newBundle])
def convertToAngles(streamline2D):
    streamline2D = np.array(streamline2D)
#     print((streamline2D))
    angles = np.zeros(streamline2D.shape[0])
    angles[0] = 1
#     print(streamline2D.shape)
    # handle the direction of start
#     print(streamline2D.shape)
    for point in range(1,streamline2D.shape[0]):
#         print(point)
        diff = streamline2D[point] - streamline2D[point-1]
#         print(streamline2D[point], streamline2D[point-1])
        if(diff[0]==0):
            angles[point] = angles[point-1]
        else:
            angles[point] = math.degrees(math.atan(diff[1]/diff[0]))
    return angles


In [22]:
# center the data
def centerData(data):
    for i in range(len(data)):
        mean = np.mean(data[i],axis=0)
        data[i] = data[i]-mean
    return data

In [23]:
def getDataLD(bundle):
    data2D = []
    angularData = []
    eigenValues = []
    eigenVecs = []
    for index in range(len(bundle)):
        newStreamline, vecs, singVal = project2D(np.array(bundle[index]))
        data2D.append(newStreamline[0][:,:2])
        eigenValues.append(singVal)
        eigenVecs.append(vecs)
        # append these values to store them 
        streamlineAsAngle = convertToAngles(newStreamline[0])
#         print(streamlineAsAngle)
        angularData.append(streamlineAsAngle)
    
    return angularData, data2D, eigenVecs, eigenValues

In [24]:
cBrain = centerData(np.array(brain))
brain1D, brain2D, eigenVecs, eigenValues = getDataLD(cBrain)
brain3D = np.array(cBrain).tolist()
# show_multiple_streamlines(newStreamline,bundle[0:1])

In [25]:
# From https://www.geeksforgeeks.org/program-to-find-equation-of-a-plane-passing-through-3-points/
# get equation of plane from 3 points
def getPlaneEquation(p1, p2, p3):    
    a1 = p2[0] - p1[0] 
    b1 = p2[1] - p1[1] 
    c1 = p2[2] - p1[2] 
    a2 = p3[0] - p1[0] 
    b2 = p3[1] - p1[1] 
    c2 = p3[2] - p1[2]
    a = b1 * c2 - b2 * c1 
    b = a2 * c1 - a1 * c2 
    c = a1 * b2 - b1 * a2 
    d = (- a * p1[0] - b * p1[1] - c * p1[2]) 
    norm = np.sqrt(a**2+b**2+c**2)
    return a/norm,b/norm,c/norm,d/norm
#     print "equation of plane is ", 
#     print a, "x +", 
#     print b, "y +", 
#     print c, "z +", 
#     print d, "= 0."


In [26]:
# brain data in a particular plane but in 3D space
x2DBrain3D = [(np.dot(eigenVecs[i].T, stream.T)).T for i, stream in enumerate(brain2D)]
# (eigenVecs[0]).shape

In [112]:
len(x2DBrain3D)

200

In [32]:
show_multiple_streamlines(x2DBrain3D[65:66],brain3D[65:66])

In [34]:
getPlaneEquation(x2DBrain3D[4][5], x2DBrain3D[4][6], x2DBrain3D[4][2])

(0.9858762976317351,
 0.0679287596567005,
 0.15308040168305262,
 0.3605892652705367)

In [35]:
def rodriguesRotation(axis, vector, angle):
    # axis should be a unit vector
    return vector*np.cos(angle) + (np.cross(axis, vector))*np.sin(angle) + axis*(np.dot(axis,vector))*(1-np.cos(angle))

In [36]:
rodriguesRotation(np.array([0,1,0]), np.array([1,0,0]), 2*np.pi)

array([1.0000000e+00, 0.0000000e+00, 2.4492936e-16])

In [37]:
atheta = 0
ztheta = 0
subSegments = 7 # using odd value to prevent redunduncy in the discrete planes
subAngles = np.pi/subSegments
noOfPlanes = subSegments*subSegments
discretePlanes = []
for a in range(subSegments):
    vec = np.array([np.cos(atheta+(a*subAngles)), np.sin(atheta+(a*subAngles)), 0])
    axis = np.array([np.cos((np.pi/2)+atheta+(a*subAngles)), np.sin((np.pi/2)+atheta+(a*subAngles)), 0])
    for z in range(subSegments):
        discretePlanes.append(rodriguesRotation(axis, vec, ztheta+(z*subAngles)))

In [38]:
discretePlanes = np.array(discretePlanes)

In [39]:
# (brain3D[0]).shape
# temp = Streamlines([(np.hstack((brain2D[3],np.zeros((brain2D[3].shape[0])).reshape((-1,1)))))])
# show_multiple_streamlines(temp,tempStream)
# print(len(brain1D),len(brain2D),len(brain3D))

In [40]:
np.dot((eigenVecs[3][1]), (eigenVecs[3][2]))

IndexError: index 2 is out of bounds for axis 0 with size 2

In [41]:
distanceBetweenCurves(brain3D[3], brain3D[81], 'fastdtw')

27224.70919387607

In [53]:
# create covariance matrix

cov = np.zeros((len(brain3D), len(brain3D)))
for i in range(len(brain3D)):
    for j in range(len(brain3D)):
        cov[i][j] = distanceBetweenCurves(brain3D[i], brain3D[j], 'fastdtw')
    print(i, end=" ")

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 

In [57]:
# store the covariance matrix variable to disk

# Save
# with open('covarianceDTW3D.pkl', 'wb') as f:
#     pickle.dump(cov, f)

In [121]:
# Retrieve
cov = []
with open('covarianceDTW3D.pkl', 'rb') as f:
    cov = pickle.load(f)

In [122]:
cov.shape

(200, 200)

In [123]:
import matplotlib.pyplot as plt
%matplotlib qt
plt.imshow(cov)

In [42]:
# get centroids for the bundles

%time
def getCentroids(streamlines, noOfCentroids, pointsInCentroid=400):
    streamlines = [set_number_of_points(streamline, nb_points=pointsInCentroid) for streamline in streamlines]
    thresh=1
    qb = QuickBundles(threshold=thresh, metric=AveragePointwiseEuclideanMetric())
    clusters = qb.cluster(streamlines)
    length = len(clusters.centroids)
    prev = 0
    flag=False
    while(length!=noOfCentroids):
        if(length>noOfCentroids):
            if flag:
                temp = thresh
                thresh= thresh + (thresh-prev)/2.0
                prev = temp
            else:
                prev=thresh
                thresh *= 2.0
        else:
            temp = thresh
            thresh = prev+(thresh-prev)/2.0
            flag = True
        qb = QuickBundles(threshold=thresh, metric=AveragePointwiseEuclideanMetric())
        clusters = qb.cluster(streamlines)
        length = len(clusters)
    return np.array(clusters.centroids)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 14.1 µs


In [54]:
%time
def getAllCentroids(brainData, labels, noOfBundles, noOfCentroids=3):
    centroids = []
    for n in range(noOfBundles):
        print(n)
        if n==0:
            c = getCentroids(brainData[labels==n], noOfCentroids)
            centroids = c.reshape((1,)+c.shape)
        else:
            centroids = np.vstack((centroids, getCentroids(brainData[labels==n], noOfCentroids).reshape((1,)+c.shape)))
    return centroids

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


In [55]:
(np.unique(labels))

array([0., 1., 2., 3.])

In [56]:
# shape = (no of bundles, no of centroids in each bundle, no of points in each centroid, dimensions of the points)
centroidMat = getAllCentroids(np.array(brain3D),labels, len(np.unique(labels)))

# Save
with open('centroidMat.pkl', 'wb') as f:
    pickle.dump(centroidMat, f)

0
1
2
3


In [126]:
# Retrieve
with open('centroidMat.pkl', 'rb') as f:
    centroidMat = pickle.load(f)

In [57]:
centroidMat.shape

(4, 3, 400, 3)

In [58]:
# get distance of a streamline from all the centroids
def getDistToCentroids(streamline):
    global centroidMat
    distMat = np.zeros((centroidMat.shape[0], centroidMat.shape[1]))
    for i in range(distMat.shape[0]):
        for j in range(distMat.shape[1]):
            distMat[i][j] = distanceBetweenCurves(centroidMat[i][j], streamline, 'fastdtw')
    return distMat

In [59]:
# get probability of getting a bundle given a streamline(shape)
def pB_S(streamline, metric='min'):
    distMat = getDistToCentroids(streamline)
    if metric=='min':
        distToBundles = np.min(distMat, axis=1)
    elif metric=='avg':
        distToBundles = np.mean(distMat, axis=1)
    
    probabilities = 1/distToBundles
    probabilities /= np.sum(probabilities)
    return probabilities

In [61]:
count=[]
predictions1 = []
predictions2 = []
predictions3 = []
predictions0 = []
for i in range(200):
    cls = ((pB_S(brain3D[i])))
    if np.argmax(cls)==0:
        predictions0.append(brain3D[i])
    if np.argmax(cls)==1:
        predictions1.append(brain3D[i])
    if np.argmax(cls)==2:
        predictions2.append(brain3D[i])
    if np.argmax(cls)==3:
        predictions3.append(brain3D[i])
    count.append(np.argmax(cls))
    print(cls)

[0.75134319 0.10062288 0.0891755  0.05885844]
[0.53317047 0.17721768 0.16244019 0.12717166]
[0.11144387 0.41299634 0.1763508  0.29920899]
[0.11832459 0.41054582 0.17915182 0.29197777]
[0.11694108 0.42199461 0.17256388 0.28850043]
[0.118765   0.42439587 0.17144236 0.28539677]
[0.58466736 0.1242622  0.19696202 0.09410842]
[0.67460314 0.12463348 0.12408988 0.07667351]
[0.08287853 0.51950108 0.14236462 0.25525576]
[0.09670777 0.47795795 0.15851169 0.26682259]
[0.6140183  0.15217763 0.13915601 0.09464805]
[0.69320814 0.11693956 0.11698009 0.07287221]
[0.11226416 0.43948818 0.17159344 0.27665422]
[0.67043385 0.11211835 0.13548577 0.08196203]
[0.09598884 0.30317022 0.16676709 0.43407385]
[0.11783129 0.41050737 0.17923861 0.29242273]
[0.13873674 0.25496204 0.23768382 0.3686174 ]
[0.54694623 0.17413954 0.15987942 0.11903481]
[0.66928853 0.12642303 0.11731032 0.08697812]
[0.53555149 0.17365244 0.17255625 0.11823982]
[0.64278108 0.13957939 0.13057071 0.08706881]
[0.09516657 0.35173957 0.16599368 

[0.0255236  0.08313855 0.05590694 0.83543091]
[0.37792632 0.20188044 0.2591792  0.16101404]
[0.38610603 0.19107409 0.26095128 0.1618686 ]
[0.37447553 0.19399661 0.2766444  0.15488346]
[0.03295691 0.09901014 0.07433116 0.79370179]
[0.00347084 0.01100555 0.00757659 0.97794702]
[0.02846297 0.08521933 0.06277302 0.82354468]
[0.02398623 0.0758253  0.05176602 0.84842246]
[0.38006589 0.19655588 0.25993462 0.16344361]
[0.37103552 0.20307105 0.26864712 0.15724631]
[0.01289393 0.03981532 0.02853331 0.91875744]
[0.37512871 0.20143376 0.2603664  0.16307112]
[0.35284633 0.21613125 0.27898049 0.15204193]
[0.01233935 0.03651166 0.0274156  0.92373338]
[0.02240334 0.06686921 0.05001063 0.86071682]
[0.00984311 0.01402483 0.02183672 0.95429534]
[0.3843752  0.19326342 0.26135658 0.16100481]
[0.3755362  0.2013591  0.2605707  0.16253401]
[0.02655418 0.09107233 0.05599779 0.8263757 ]
[0.38004643 0.19737157 0.26255079 0.16003121]
[0.36740709 0.18376189 0.27222059 0.17661043]


In [226]:
show_streamlines(predictions)

/Users/rp7/anaconda3/lib/python3.6/site-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \


In [73]:
count = np.array(count)
(np.sum(count[:50]==0) + np.sum(count[50:100]==1) + np.sum(count[100:150]==2) + np.sum(count[150:]==3))/200


0.485

In [74]:
# probability of a centroid given a particular streamline(shape)
# dimensions: (no of bundles, no of centroids)
def pC_S(streamline):
    dist = 1/getDistToCentroids(streamline)
    return dist/np.sum(dist)

In [75]:
# centroidMat.reshape((2,3)).shape
# (np.dot(eigenVecs[i].T, stream.T)).T
len(discretePlanes)

49

In [76]:
# get angle between 2 planes
def dihedralAngle(a, b):
    temp =  np.arccos((np.dot(a,b))/(np.linalg.norm(a)*np.linalg.norm(b)))
    if temp>10000:
        print(a,b)
    return temp


In [77]:
# dimensions: (total number of centroids, number of discrete planes)
def pP_C():
    noOfPlanes = len(discretePlanes)
    probabilities = np.zeros((centroidMat.shape[0]*centroidMat.shape[1], noOfPlanes))
    for i in range(centroidMat.shape[0]):
        for j in range(centroidMat.shape[1]):
            cent, evecs, evalue = project2D(centerData(centroidMat[i][j]))
            PCAedCentroid = (np.dot(evecs.T, np.array(cent[0]).T)).T
            a,b,c,d = getPlaneEquation(PCAedCentroid[0],PCAedCentroid[1],PCAedCentroid[2])
            for k in range(noOfPlanes):
                probabilities[centroidMat.shape[1]*i+j][k] = dihedralAngle(np.array([a,b,c]), discretePlanes[k])
            
    probabilities = 1.0/probabilities
    probabilities /= np.sum(probabilities, axis=1).reshape((-1,1))
    return probabilities
    

In [78]:
dihedralAngle(np.array([-0.5775682004691819, -0.5773971386380066, -0.5770853646553805]), discretePlanes[4])

0.8077343472643393

In [79]:
pPC = pP_C()

In [80]:
centroidMat.shape

(4, 3, 400, 3)

In [81]:
# dimensions: (number of discrete planes, no of bundles)
def pB_P(pPC, metric='avg'):
    noOfPlanes = len(discretePlanes)
    angles = np.zeros((centroidMat.shape[0]*centroidMat.shape[1], noOfPlanes))
#     centDist = np.sum(pPC, axis=1)
    for i in range(centroidMat.shape[0]):
        for j in range(centroidMat.shape[1]):
            cent, evecs, evalue = project2D(centerData(centroidMat[i][j]))
            PCAedCentroid = (np.dot(evecs.T, np.array(cent[0]).T)).T
            a,b,c,d = getPlaneEquation(PCAedCentroid[0],PCAedCentroid[1],PCAedCentroid[2])
            for k in range(noOfPlanes):
                angles[centroidMat.shape[1]*i+j][k] = dihedralAngle(np.array([a,b,c]), discretePlanes[k])
    probabilities = np.zeros((noOfPlanes, centroidMat.shape[0]))
    for i in range(probabilities.shape[1]):
        probabilities[:,i] = np.sum(angles[i*centroidMat.shape[1]:(i+1)*centroidMat.shape[1],:],axis=0)
#     print(np.sum(probabilities, axis=1).shape)
    probabilities = 1.0/probabilities
    probabilities /= np.sum(probabilities, axis=1).reshape((-1,1))
    
    return probabilities
    

In [82]:
# probability of Bundle given centroid and plane information
# dimensions: ((no of centroids*no of discrete planes), no of bundles)
def pB_PC(pBP, plane, centroid):
    return pBP[plane]*pPC[centroid][plane]

In [83]:
pBP = pB_P(pPC)

In [85]:
def predict(streamline):
    l = np.argmax(np.random.random(centroidMat.shape[0]))
    answers = []
    probc = pC_S(streamline).flatten()
    prevb = 0
    for i in range(5000):
        c = np.random.choice(np.arange(probc.shape[0]), p=probc)
        probp = pPC[c,:]
        p = np.random.choice(np.arange(probp.shape[0]), p=probp)
        prob = pB_PC(pBP, p, c)
        finalProb = prob*probc[c]*probp[p]
        finalProb/=np.sum(finalProb)
        l = (np.random.choice(np.arange(finalProb.shape[0]), p=finalProb))
        answers.append(l)
    return (np.bincount(answers[4000:]))

In [ ]:
acc = []
for i in range(200):
    acc.append(np.argmax(predict(brain[i])))
#     print(acc)

In [ ]:
acc = np.array(acc)
(np.sum(acc[:50]==0) + np.sum(acc[50:100]==1) + np.sum(acc[100:150]==2) + np.sum(acc[150:]==3))/200


In [ ]:
#show_streamlines(cst)

In [ ]:
#show_multiple_streamlines(bundle,cst)

let's see its data type

In [ ]:
#bundle # whole bundle, made of several streamlines

In [ ]:
# ((brain)[0]).shape# number of streamlines in bundle

In [ ]:
# bundle[1] #one streamline in bundle
# (np.array(bundle)[0]).shape
# np.array(bundle)

In [ ]:
# (bundle[1].shape) #one point on one streamline

In [117]:
npbrain = np.array(brain)

In [ ]:
# npbrain[0].shape

In [ ]:
# len(bundle)

these are x, y and z co-ordinates of the point

In [118]:
# get labels

def getLabels(brain, bundle):
    labels = np.zeros(len(brain))
    npbrain = np.array(brain)
    for i in range(len(bundle)):
        for j in range(len(brain)):
            if((npbrain[j].shape[0]==bundle[i].shape[0])):
    #         print(i,j)
                if np.allclose(npbrain[j], bundle[i]):
                    labels[j] = 1
                    break
        if(i%100==0):
            print(i,end=" ")
    return labels

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 

In [21]:
# npbrain[0].shape


In [23]:
n_pts = [len(streamline) for streamline in bundle]

In [ ]:
from dipy.tracking.streamline import (set_number_of_points, nbytes,select_random_set_of_streamlines)
newStreamlines = set_number_of_points(bundle,101)
n_pts_ds = [len(streamline) for streamline in newStreamlines]

In [ ]:
newBrain = set_number_of_points(brain,101)

In [127]:
# newBrain[0].shape
trainset[0].shape

(619,)

In [119]:
# len(npbrain)
npbrain = np.array(brain1D)

In [120]:
import tensorflow as tf
import keras

In [121]:
# ==========
#   MODEL
# ==========

# Parameters
learning_rate = 0.02
training_steps = 70
batch_size = 100
display_step = 200

# Network Parameters
seq_max_len = 1000 # Sequence max length
n_hidden = 64 # hidden layer num of features
n_classes = 2 # linear sequence or not

trainset = npbrain[1950:2050]
testset = labels[1950:2050]

# tf Graph input
x = tf.placeholder("float", [None, seq_max_len, 3])
y = tf.placeholder("float", [None, n_classes])
# A placeholder for indicating each sequence length
seqlen = tf.placeholder(tf.int32, [None])

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [122]:
def dynamicRNN(x, seqlen, weights, biases):
    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, seq_max_len, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden)

    # Get lstm cell output, providing 'sequence_length' will perform dynamic
    # calculation.
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, x, dtype=tf.float32,
                                sequence_length=seqlen)

    # When performing dynamic calculation, we must retrieve the last
    # dynamically computed output, i.e., if a sequence length is 10, we need
    # to retrieve the 10th output.
    # However TensorFlow doesn't support advanced indexing yet, so we build
    # a custom op that for each sample in batch size, get its length and
    # get the corresponding relevant output.

    # 'outputs' is a list of output at every timestep, we pack them in a Tensor
    # and change back dimension to [batch_size, n_step, n_input]
    outputs = tf.stack(outputs)
    outputs = tf.transpose(outputs, [1, 0, 2])

    # Hack to build the indexing and retrieve the right output.
    batch_size = tf.shape(outputs)[0]
    # Start indices for each sample
    index = tf.range(0, batch_size) * seq_max_len + (seqlen - 1)
    # Indexing
    outputs = tf.gather(tf.reshape(outputs, [-1, n_hidden]), index)

    # Linear activation, using outputs computed above
    return tf.matmul(outputs, weights['out']) + biases['out']

In [123]:
tf_session = keras.backend.get_session()
tf_graph = tf.get_default_graph()
# tf.reset_default_graph()
print([n.name for n in tf.get_default_graph().as_graph_def().node])
# tf.contrib.rnn.reset_states()
# with tf_session.as_default():
#      with tf_graph.as_default():
#             tf.reset_default_graph() 
pred = dynamicRNN(x, seqlen, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

['Placeholder', 'Placeholder_1', 'Placeholder_2', 'random_normal/shape', 'random_normal/mean', 'random_normal/stddev', 'random_normal/RandomStandardNormal', 'random_normal/mul', 'random_normal', 'Variable', 'Variable/Assign', 'Variable/read', 'random_normal_1/shape', 'random_normal_1/mean', 'random_normal_1/stddev', 'random_normal_1/RandomStandardNormal', 'random_normal_1/mul', 'random_normal_1', 'Variable_1', 'Variable_1/Assign', 'Variable_1/read', 'IsVariableInitialized', 'IsVariableInitialized_1', 'init']
Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



/Users/rp7/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [124]:
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, training_steps+1):
#         batch_x, batch_y, batch_seqlen = trainset.next(batch_size)
        rangelow = ((step-1)*batch_size)%(trainset.shape[0])
        rangehigh = ((step)*batch_size)%(trainset.shape[0])
        if(rangehigh==rangelow):
            rangehigh=rangehigh-1
        if(rangehigh<rangelow):
            batch_x = np.append(trainset[rangelow:], trainset[:rangehigh])
            batch_y = np.append(testset[rangelow:], testset[:rangehigh])
        else:
            batch_x = trainset[rangelow:rangehigh]
            batch_y = testset[rangelow:rangehigh]
        
        batch_seqlen = []
        for sample in trainset:
            batch_seqlen.append(sample.shape[0])

#         batch_seqlen = batch_seqlen.tolist()
        print(type(batch_seqlen))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       seqlen: batch_seqlen})
        if step % display_step == 0 or step == 1:
            # Calculate batch accuracy & loss
            acc, loss = sess.run([accuracy, cost], feed_dict={x: batch_x, y: batch_y,
                                                seqlen: batch_seqlen})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy
    test_data = testset.data
    test_label = testset.labels
    test_seqlen = testset.seqlen
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label,
                                      seqlen: test_seqlen}))

<class 'list'>


ValueError: setting an array element with a sequence.

In [ ]:
class ToySequenceData(object):
    """ Generate sequence of data with dynamic length.
    This class generate samples for training:
    - Class 0: linear sequences (i.e. [0, 1, 2, 3,...])
    - Class 1: random sequences (i.e. [1, 3, 10, 7,...])

    NOTICE:
    We have to pad each sequence to reach 'max_seq_len' for TensorFlow
    consistency (we cannot feed a numpy array with inconsistent
    dimensions). The dynamic calculation will then be perform thanks to
    'seqlen' attribute that records every actual sequence length.
    """
    def __init__(self, n_samples=1000, max_seq_len=20, min_seq_len=3,
                 max_value=1000):
        self.data = []
        self.labels = []
        self.seqlen = []
        for i in range(n_samples):
            # Random sequence length
            len = np.random.randint(min_seq_len, max_seq_len)
            # Monitor sequence length for TensorFlow dynamic calculation
            self.seqlen.append(len)
            # Add a random or linear int sequence (50% prob)
            if np.random.random() < .5:
                # Generate a linear sequence
                rand_start = np.random.randint(0, max_value - len)
                s = [[float(i)/max_value] for i in
                     range(rand_start, rand_start + len)]
                # Pad sequence for dimension consistency
                s += [[0.] for i in range(max_seq_len - len)]
                self.data.append(s)
                self.labels.append([1., 0.])
            else:
                # Generate a random sequence
                s = [[float(np.random.randint(0, max_value))/max_value]
                     for i in range(len)]
                # Pad sequence for dimension consistency
                s += [[0.] for i in range(max_seq_len - len)]
                self.data.append(s)
                self.labels.append([0., 1.])
        self.batch_id = 0

    def next(self, batch_size):
        """ Return a batch of data. When dataset end is reached, start over.
        """
        if self.batch_id == len(self.data):
            self.batch_id = 0
        batch_data = (self.data[self.batch_id:min(self.batch_id +
                                                  batch_size, len(self.data))])
        batch_labels = (self.labels[self.batch_id:min(self.batch_id +
                                                  batch_size, len(self.data))])
        batch_seqlen = (self.seqlen[self.batch_id:min(self.batch_id +
                                                  batch_size, len(self.data))])
        print(type(batch_seqlen[0]))
        self.batch_id = min(self.batch_id + batch_size, len(self.data))
        return batch_data, batch_labels, batch_seqlen

In [ ]:
tr = ToySequenceData(n_samples=1000, max_seq_len=seq_max_len)
te = ToySequenceData(n_samples=500, max_seq_len=seq_max_len)

In [ ]:
b_x, b_y, b_seqlen = tr.next(100)

In [ ]:
print((b_x))
# print(batch_seqlen)